In [1]:
from vespa.package import *
from pathlib import Path
import json

app_package = ApplicationPackage(
    name="wiki",
    components=[
        Component(
            id="translation",
            type="hugging-face-embedder",
            parameters=[
                Parameter("encoder", {"path": "model/encoder_model.onnx"}),
                Parameter("decoder", {"path": "model/decoder_model_merged.onnx"}),
            ],
        )
    ],
)

In [ ]:
app_package.schema.add_fields(
    Field(name="id", type="int", indexing=["attribute", "summary"]),
    Field(name="title", type="string", indexing=["index", "summary"], index="enable-bm25"),
    Field(name="url", type="string", indexing=["index", "summary"], index="enable-bm25"),
    Field(name="paragraphs", type="array<string>", indexing=["index", "summary"],
          index="enable-bm25", bolding=True),

In [4]:
from optimum.pipelines import pipeline

# The model will be loaded to an ORT.
onnx_trans = pipeline(
    "text2text-generation", model="Helsinki-NLP/opus-mt-en-fr", accelerator="ort"
)

Framework not specified. Using pt to export the model.
/workspaces/vespa-client/optimum/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
/workspaces/vespa-client/optimum/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install 

In [7]:
onnx_trans.save_pretrained("model-mt", monolith=True)

TypeError: Pipeline.save_pretrained() got an unexpected keyword argument 'monolith'